In [1]:
from facebookads.adobjects.customaudience import CustomAudience
from facebookads.adobjects.adaccount import AdAccount
from facebookads.api import FacebookAdsApi
import json
import pandas as pd
import os
import datetime
import numpy as np
import glob
import time
import paramiko

BigLotsAccount_id="act_271491453638620"
BigLotsDcart10_id="act_2449979771956630"

list_account_id=[BigLotsAccount_id,BigLotsDcart10_id]
# jian_APP_ID="2537704939796694"# jian's fb marketing app id
jian_token=json.load(open("/mnt/clients/juba/hqjubaapp02/sharefolder/Docs/FB_token/lasest_FB_token_jian.json","r"))['latest_FB_token']



In [2]:
def recursive_file_gen(root_path):
    for root, dirs, files in os.walk(root_path):
        for file in files:
            yield os.path.join(root,file)

In [3]:
list_already_copied_deltas=list(recursive_file_gen("./DeltaFiles/"))
list_already_copied_deltas

['./DeltaFiles/File_20201125/Near_Lapsing_Delta__20201125080541.csv',
 './DeltaFiles/File_20201125/Reactivation_Redeemers__20201125082106.csv',
 './DeltaFiles/File_20201125/13_Month_Lapsed_Delta__20201125080533.csv',
 './DeltaFiles/File_20201125/Signup_NoPurchase_Delta__20201125080540.csv',
 './DeltaFiles/File_20201209/Near_Lapsing_Delta__20201209080443.csv',
 './DeltaFiles/File_20201209/Reactivation_Redeemers__20201209082351.csv',
 './DeltaFiles/File_20201209/13_Month_Lapsed_Delta__20201209080749.csv',
 './DeltaFiles/File_20201209/Signup_NoPurchase_Delta__20201209080747.csv',
 './DeltaFiles/File_20201223/Near_Lapsing_Delta__20201223080702.csv',
 './DeltaFiles/File_20201223/13_Month_Lapsed_Delta__20201223080705.csv',
 './DeltaFiles/File_20201223/Reactivation_Redeemers__20201223082349.csv',
 './DeltaFiles/File_20201223/Signup_NoPurchase_Delta__20201223080705.csv',
 './DeltaFiles/File_20201216/Near_Lapsing_Delta__20201216080348.csv',
 './DeltaFiles/File_20201216/Reactivation_Redeemers__2

In [4]:
remote_data_path_client="/home/agilone/"

host = "107.191.32.220" #hard-coded
port = 22
transport = paramiko.Transport((host, port))

password = "Tur87ZqzF9xV5mYF" #hard-coded
username = "agilone" #hard-coded
transport.connect(username = username, password = password)
sftp = paramiko.SFTPClient.from_transport(transport)


In [5]:
list_remote_new_delta_file=sftp.listdir()

list_remote_new_delta_file=[x for x in list_remote_new_delta_file if "_Delta_" in x or "Reactivation_Redeem" in x]
list_remote_new_delta_file=[x for x in list_remote_new_delta_file if "subscrib" not in x.lower()]
list_remote_new_delta_file

['Signup_NoPurchase_Delta__20210106080650.csv',
 '13_Month_Lapsed_Delta__20210106080645.csv',
 'Reactivation_Redeemers__20210106082111.csv',
 'Near_Lapsing_Delta__20210106080652.csv']

In [6]:
date=set([x.split("__")[1][:8] for x in list_remote_new_delta_file])
if len(date)==1:
    date=list(date)[0]
else:
    raise ValueError("multiple dates")
print(date)

20210106


In [7]:
local_folder_new_files_delta="./DeltaFiles/File_%s/"%date
remote_movein_folder="/home/agilone/audience_10_off_20/delta/"

try:
    os.stat(local_folder_new_files_delta)
except:
    os.mkdir(local_folder_new_files_delta)


if len(list_remote_new_delta_file)==4:
    for file in list_remote_new_delta_file:
        local_file=local_folder_new_files_delta+os.path.basename(file)
        sftp.get(file,local_file)
        sftp.rename(file, remote_movein_folder+os.path.basename(file))      
else:
    raise ValueError("new received files not 4")

In [8]:
list_files_initial_test=glob.glob("./InitialFiles/*.csv")
list_files_initial_test=[x for x in list_files_initial_test if "A_" in os.path.basename(x) or "B_" in os.path.basename(x)]
list_files_initial_test.sort()
df_initial_test=pd.DataFrame()
for file in list_files_initial_test:
    df=pd.read_csv(file,dtype=str)
    df['segment']=os.path.basename(file).split(".")[0]
    df_initial_test=df_initial_test.append(df)
print(df_initial_test.shape)
df_initial_test.groupby('segment')['customersummary_mastercustomerid','customersummary_emailhash','customersummary_c_primaryscnhash'].nunique().reset_index()

(10472244, 5)


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  # Remove the CWD from sys.path while we load stuff.


,segment,customersummary_mastercustomerid,customersummary_emailhash,customersummary_c_primaryscnhash
0,10-12 Month About to Lapse MS A_20201102075436,2405635,2405575,2405635
1,10-12 Month About to Lapse MS B_20201102075837,223906,223906,223906
2,13-18 Month Lapsed MS A_20201102073523,3541148,3541019,3541148
3,13-18 Month Lapsed MS B_20201102073920,341226,341224,341226
4,Online Signup MS A_20201102074635,3600659,3600595,3600659
5,Online Signup MS B_20201102074942,359670,359666,359670


In [9]:
df_initial_test.head(2)

,customersummary_mastercustomerid,customersummary_emailhash,customersummary_c_primaryscnhash,execution_id,segment
0,KFK_0_58796144,25abf112b00caffcbd9aef32d66bd67a80b853ce039dcb...,068b3d7bc1f50214e5f0a6e05b8565359687334cddbcf2...,1604346769164,10-12 Month About to Lapse MS A_20201102075436
1,KFK_0_10898437,976d2bcdbab584263442ea5cc4fb28c0d318f13a4be6b4...,af002365ba37f81669d9db9b61b2f6c5a2f6e3bf4f2bbc...,1604346769164,10-12 Month About to Lapse MS A_20201102075436


In [10]:
all_redeemers=list(recursive_file_gen("./DeltaFiles/"))
all_redeemers=[x for x in all_redeemers if "Redeemers" in x]
df_redeemers=pd.DataFrame()

for file in all_redeemers:
    df=pd.read_csv(file,dtype=str).drop_duplicates()
    df_redeemers=df_redeemers.append(df)
df_redeemers.shape,df_redeemers['customersummary_emailhash'].nunique(),df_redeemers['customersummary_c_primaryscnhash'].nunique(),df_redeemers['customersummary_c_primaryscnhash'].nunique()
  

((411109, 4), 78142, 78118, 78118)

In [11]:
del df_redeemers['execution_id']
df_redeemers=df_redeemers.drop_duplicates()
df_redeemers.shape,df_redeemers['customersummary_emailhash'].nunique(),df_redeemers['customersummary_c_primaryscnhash'].nunique(),df_redeemers['customersummary_c_primaryscnhash'].nunique()


((79776, 3), 78142, 78118, 78118)

In [12]:
df_initial_test['recent_redeem_status']=np.where(df_initial_test['customersummary_mastercustomerid'].isin(df_redeemers['customersummary_mastercustomerid'].tolist()),"redeemers",
                                    np.where(df_initial_test['customersummary_emailhash'].isin(df_redeemers['customersummary_emailhash'].tolist()),"redeemers",
                                             np.where(df_initial_test['customersummary_c_primaryscnhash'].isin(df_redeemers['customersummary_c_primaryscnhash'].tolist()),"redeemers",
                                                      "no_redeemed_yet"
                                                     )
                                            )
                                    )
df_initial_test.head(2)

,customersummary_mastercustomerid,customersummary_emailhash,customersummary_c_primaryscnhash,execution_id,segment,recent_redeem_status
0,KFK_0_58796144,25abf112b00caffcbd9aef32d66bd67a80b853ce039dcb...,068b3d7bc1f50214e5f0a6e05b8565359687334cddbcf2...,1604346769164,10-12 Month About to Lapse MS A_20201102075436,no_redeemed_yet
1,KFK_0_10898437,976d2bcdbab584263442ea5cc4fb28c0d318f13a4be6b4...,af002365ba37f81669d9db9b61b2f6c5a2f6e3bf4f2bbc...,1604346769164,10-12 Month About to Lapse MS A_20201102075436,no_redeemed_yet


In [13]:
df_output_performance=df_initial_test.groupby(["segment",'recent_redeem_status'])['customersummary_mastercustomerid'].count().reset_index()

df_output_performance=df_output_performance.pivot(index="segment",columns="recent_redeem_status",values="customersummary_mastercustomerid").reset_index()
df_output_performance['conv_rate']=df_output_performance['redeemers']/(df_output_performance['redeemers']+df_output_performance['no_redeemed_yet'])
df_output_performance

recent_redeem_status,segment,no_redeemed_yet,redeemers,conv_rate
0,10-12 Month About to Lapse MS A_20201102075436,2380293,25342,0.010534
1,10-12 Month About to Lapse MS B_20201102075837,223805,101,0.000451
2,13-18 Month Lapsed MS A_20201102073523,3524719,16429,0.004639
3,13-18 Month Lapsed MS B_20201102073920,341199,27,0.000079
4,Online Signup MS A_20201102074635,3573139,27520,0.007643
5,Online Signup MS B_20201102074942,357929,1741,0.004841


In [14]:
df_output_performance['redeemers'].sum()

71160

In [15]:
df_output_performance.to_csv("./BL_10_off_20_performance_as_initial_up_to_%s_JL_%s.csv"%(local_folder_new_files_delta[-9:-1],str(datetime.datetime.now().date())),index=False)


In [16]:
dict_Audience_ID={"Near_Lapsing":[23845836194910265,23846192280570632],
                 '13_Month_Lapsed':[23845836216340265,23846192299680632],
                 'Signup_NoPurchase':[23845836247620265,23846192327750632]}

list_all_ids=[]
for v in dict_Audience_ID.values():
    list_all_ids.extend(v)
# test A only with two FB account (general and dcart10)
list_all_ids


[23845836194910265,
 23846192280570632,
 23845836216340265,
 23846192299680632,
 23845836247620265,
 23846192327750632]

In [17]:
def remove_ids_from_account(token,aud_id,list_ids_to_remove,fb_schema=CustomAudience.Schema.email_hash):
    FacebookAdsApi.init(access_token=token, api_version='v8.0')
    audience = CustomAudience(aud_id)
    listlen=len(list_ids_to_remove)
    
    chunck_size=1000
    count_removed=0
    for i in range(int(np.ceil(listlen/chunck_size))):
        starti = i*chunck_size
        if (i+1)*chunck_size<listlen:
            endi = (i+1)*chunck_size
        else:
            endi = listlen
        list0 = list_ids_to_remove[starti:endi]
        
        count_removed+=len(list0)
        audience.remove_users(fb_schema, list0)
        time.sleep(1)
    print("%i IDs removed from the audience id: %i"%(count_removed,aud_id))
    
def add_ids_to_account(token,aud_id,list_ids_to_add,fb_schema=CustomAudience.Schema.email_hash):
    FacebookAdsApi.init(access_token=token, api_version='v8.0')
    audience = CustomAudience(aud_id)
    listlen=len(list_ids_to_add)
    
    chunck_size=30000
    count_added=0
    for i in range(int(np.ceil(listlen/chunck_size))):
        starti = i*chunck_size
        if (i+1)*chunck_size<listlen:
            endi = (i+1)*chunck_size
        else:
            endi = listlen
        list0 = list_ids_to_add[starti:endi]
        
        count_added+=len(list0)
        audience.add_users(fb_schema, list0)
    print("%i IDs added to the audience id: %i"%(count_added,aud_id))
    


In [18]:
all_redeemers.sort()
new_redeemer=all_redeemers[-1]

df_new_redeemers=pd.read_csv(new_redeemer,usecols=["customersummary_emailhash"],dtype=str)


In [19]:
# remove all of the redeemers from each of the test ids

for fb_id in list_all_ids:
    list_emails_to_remove=df_redeemers['customersummary_emailhash'].unique().tolist()
    list_emails_to_remove=[x for x in list_emails_to_remove if len(x)==64] #SHA256 check
    remove_ids_from_account(token=jian_token,
                            aud_id=fb_id,
                            list_ids_to_remove=list_emails_to_remove,
                            fb_schema=CustomAudience.Schema.email_hash)

78140 IDs removed from the audience id: 23845836194910265
78140 IDs removed from the audience id: 23846192280570632
78140 IDs removed from the audience id: 23845836216340265
78140 IDs removed from the audience id: 23846192299680632
78140 IDs removed from the audience id: 23845836247620265
78140 IDs removed from the audience id: 23846192327750632


In [20]:
list_new_deltas=glob.glob(local_folder_new_files_delta+"*.csv")
list_new_deltas

['./DeltaFiles/File_20210106/Near_Lapsing_Delta__20210106080652.csv',
 './DeltaFiles/File_20210106/13_Month_Lapsed_Delta__20210106080645.csv',
 './DeltaFiles/File_20210106/Signup_NoPurchase_Delta__20210106080650.csv',
 './DeltaFiles/File_20210106/Reactivation_Redeemers__20210106082111.csv']

In [21]:
# 1 Near_Lapsing
str_pattern="Near_Lapsing"
for fb_id in dict_Audience_ID[str_pattern]:
    file=[x for x in list_new_deltas if str_pattern in x]
    print(file)
    if len(file)==1:
        list_emails_to_add=pd.read_csv(file[0],dtype=str,usecols=['customersummary_emailhash'])['customersummary_emailhash'].unique().tolist()
        list_emails_to_add=[x for x in list_emails_to_add if len(x)==64]
    else:
        raise ValueError("multiple files from list_new_deltas for %s"%str_pattern)
    add_ids_to_account(token=jian_token,
                            aud_id=fb_id,
                            list_ids_to_add=list_emails_to_add,
                            fb_schema=CustomAudience.Schema.email_hash)

['./DeltaFiles/File_20210106/Near_Lapsing_Delta__20210106080652.csv']
178465 IDs added to the audience id: 23845836194910265
['./DeltaFiles/File_20210106/Near_Lapsing_Delta__20210106080652.csv']
178465 IDs added to the audience id: 23846192280570632


In [22]:
# 2 13_Month_Lapsed
str_pattern="13_Month_Lapsed"
for fb_id in dict_Audience_ID[str_pattern]:
    file=[x for x in list_new_deltas if str_pattern in x]
    print(file)
    if len(file)==1:
        list_emails_to_add=pd.read_csv(file[0],dtype=str,usecols=['customersummary_emailhash'])['customersummary_emailhash'].unique().tolist()
        list_emails_to_add=[x for x in list_emails_to_add if len(x)==64]
    else:
        raise ValueError("multiple files from list_new_deltas for %s"%str_pattern)
    add_ids_to_account(token=jian_token,
                            aud_id=fb_id,
                            list_ids_to_add=list_emails_to_add,
                            fb_schema=CustomAudience.Schema.email_hash)

['./DeltaFiles/File_20210106/13_Month_Lapsed_Delta__20210106080645.csv']
149518 IDs added to the audience id: 23845836216340265
['./DeltaFiles/File_20210106/13_Month_Lapsed_Delta__20210106080645.csv']
149518 IDs added to the audience id: 23846192299680632


In [23]:
# 3 13_Month_Lapsed
str_pattern="Signup_NoPurchase"
for fb_id in dict_Audience_ID[str_pattern]:
    file=[x for x in list_new_deltas if str_pattern in x]
    print(file)
    if len(file)==1:
        list_emails_to_add=pd.read_csv(file[0],dtype=str,usecols=['customersummary_emailhash'])['customersummary_emailhash'].unique().tolist()
        list_emails_to_add=[x for x in list_emails_to_add if len(x)==64]
    else:
        raise ValueError("multiple files from list_new_deltas for %s"%str_pattern)
    add_ids_to_account(token=jian_token,
                            aud_id=fb_id,
                            list_ids_to_add=list_emails_to_add,
                            fb_schema=CustomAudience.Schema.email_hash)

['./DeltaFiles/File_20210106/Signup_NoPurchase_Delta__20210106080650.csv']
134319 IDs added to the audience id: 23845836247620265
['./DeltaFiles/File_20210106/Signup_NoPurchase_Delta__20210106080650.csv']
134319 IDs added to the audience id: 23846192327750632
